In [1]:
import helperfunction as hf
import matplotlib.pyplot as plt

In [2]:
invoice_df = hf.create_df('invoice_data')
invoice_df.head()

Current work Directory C:\Users\jahna\OneDrive\Documents\GitHub\Customer-Segmentation\Code
File Path is  C:\Users\jahna\OneDrive\Documents\GitHub\Customer-Segmentation\Code\invoice_data


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
print(f"\nTotal observations (records): {invoice_df.shape[0]}")
print(f"\nNumber of Invoices: {invoice_df['InvoiceNo'].nunique()}")
print(f"\nNumber of Customers: {invoice_df['CustomerID'].nunique()}")
print(f"\nNumber of Countries: {invoice_df['Country'].nunique()}")
print(f"\nMinimum Date: {invoice_df['InvoiceDate'].min()} - Maximum Date: {invoice_df['InvoiceDate'].max()}")


Total observations (records): 541909

Number of Invoices: 25900

Number of Customers: 4372

Number of Countries: 38

Minimum Date: 2010-12-01 08:26:00 - Maximum Date: 2011-12-09 12:50:00


In [4]:
invoice_df['price'] = invoice_df['Quantity']*invoice_df['UnitPrice']
a = invoice_df[['InvoiceNo','price']].groupby(['InvoiceNo']).sum().reset_index()
print(f"\nMin Basket Price: {a.price.min()}")
print(f"Max Basket Price: {a.price.max()}")


Min Basket Price: -168469.6
Max Basket Price: 168469.6


In [5]:
a = invoice_df[['CustomerID','price']].groupby(['CustomerID']).sum().reset_index()
print(a.sort_values(by='price', ascending=0).head())
print(f"\nMin Purchase by customer: {a.price.min()}")
print(f"Max Purchase customer: {a.price.max()}")

      CustomerID      price
1703     14646.0  279489.02
4233     18102.0  256438.49
3758     17450.0  187482.17
1895     14911.0  132572.62
55       12415.0  123725.45

Min Purchase by customer: -4287.63
Max Purchase customer: 279489.02


In [6]:
result = invoice_df.groupby('CustomerID')['InvoiceDate'].agg(['first', 'last'])
result['Date_Difference'] = result['last'] - result['first']
print(f"\nAvg customer span: {result['Date_Difference'].mean()}")


Avg customer span: 133 days 17:25:29.204025618


In [7]:
a = invoice_df[['InvoiceNo','Quantity']].groupby(['InvoiceNo']).sum().reset_index()
print(f"\nMin Quantity by InvoiceNo: {a.Quantity.min()}")
print(f"Max Quantity by InvoiceNo: {a.Quantity.max()}")


Min Quantity by InvoiceNo: -80995
Max Quantity by InvoiceNo: 80995


## EDA

## Product Categories

### Clustering Products 

### Silhoutte Intra cluster score

### PCA

## Customer Categories

## Classification on Customers